<a href="https://colab.research.google.com/github/david-c-brown/clinic_kpi_monthly/blob/main/ve_ut_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
# file exists locally via dropbox desktop
file_path = "dropbox/path/to/master_file.xlsm"

# inconsistent sheet names need to be normalized
clinic_master_sheet_names = ["CLIN1 PT VISITS-CAP",
                            "Clnc2 PER PT VISITS-CAPACITY",
                            "CLINIC 3 PT VISITS-CAP",
                            "Clinic4 PT VISITS-CAPACITY",
                            ]
# normalization dictionary
location_mapping = {
    'CLIN1': 'Clinic 1',
    'Clnc2': 'Clinic 2',
    'CLINIC 3': 'Clinic 3',
    'Clinic4': 'Clinic 4'
}

# this function cleans all the sheet names to match with our other output files
def get_location_name(name):
    if "PER" in name:
        location = name.split("PER")[0].strip()
    elif "PT" in name:
        location = name.split("PT")[0].strip()
    else:
        location = name
    if location in location_mapping:
        location = location_mapping[location]
    else:
        location = location.title()
    return location


In [ ]:
# establish columns ahead of time. these have been standardized across the sheets
result_df = pd.DataFrame(index = ['Total Visits', 'Utilization', 'Visits/Eval', 'Rev/Visit', 'Revenue'])

for sheet in tqdm(clinic_master_sheet_names):
    df = pd.read_excel(file_path, sheet_name=sheet, header=None)

    # find the row where 'Monthly Metrics' is located
    monthly_metrics_row = df[df.eq('Monthly Metrics').any(1)].index[0]

    # set this row as the header for the DataFrame
    df.columns = df.iloc[monthly_metrics_row]

    # start the DataFrame after the header row
    df = df.iloc[monthly_metrics_row+1:monthly_metrics_row+16]

    # set the index to be 'Monthly Metrics' column
    df.set_index('Monthly Metrics', inplace=True)

    # select the desired rows and columns
    df = df.loc[['Total Visits', 'Utilization', 'Visits/Eval','Rev/Visit', 'Revenue'], :'Clinic'].iloc[:, :-1]
    # df = df.dropna(axis=1) # this is no longer needed

    sheet = get_location_name(sheet)
    # add a new row with the sheet name
    df_sheet = pd.DataFrame([sheet]*df.shape[1], index=df.columns).T
    df_sheet.index = ['Clinic']
    df = pd.concat([df_sheet, df])

    result_df = pd.concat([result_df, df], axis = 1)
    # allows monitoring during process
    clear_output()
    print(sheet)

result_df = result_df.transpose().reset_index().rename(columns={'index': 'First Name'})

result_df.to_csv('per_pt_clinic_master.csv')
